DATARESEARCH2: 
EN este documento nos centraremos en obtener tablas importantes para nuestra seleccion de paises, la inflacion, o deflactor del PIB, es un factor precursor de politicas a la larga negativas para el bienestar del individuo,
tambien se puede relacionar con el PIB PER CAPITA, el SMI y el Sueldo bruto Anual Medio.
Vamos a relacionar datos economicos con otros factores importantes a la hora de agrupar paises como la zona geografica y similitudes culturales.
Voy a utilizar Selenium com ChromeDriver instalado en mi ordenador de sobremesa, pero ahora mismo hay una extension de chrome , en la web de chrome de extensiones llamada "Instant Data Scrapper" que ofrece la posibilidad de descargar las tablas en formato
Excel y CSV, a tiempo real, sin ningun tipo de explorador configurado para tareas automatizadas.
La primera tabla que obtendremos sera el indicador de Inflacion-Deflactor del pib europeo por paises.
La segunda tabla que obtendremos sera el pib per capita europeo por paises.
La tercera tabla sera el sueldo bruto anual medio europeo por paises.

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.service import Service 
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from bs4 import BeautifulSoup as bs
import requests
import random 
import pandas as pd
import numpy as np
import sqlite3
import time
import os
import pathlib

In [ ]:
#Procedemos a descargar la tabla de Inflacion en la zona euro del 2023, dado que la del año 2024 aun no esta disponible.
driver = webdriver.Chrome()
url = "https://datos.bancomundial.org/indicador/NY.GDP.DEFL.KD.ZG?contextual=region&end=2023&locations=EU-DK&most_recent_value_desc=false&start=1961&view=chart"
#Nuestra tabla esta anidada en un div de esta forma:
#<div><div class="item" style="min-height: 58px;"><div data-customlink="nl:body content" data-text="Dinamarca">ent_value<a class="country-name" 
#href="/indicador/NY.GDP.DEFL.KD.ZG?contextual=region&amp;end=2023&amp;locations=EU-DK&amp;most_rec_desc=false&amp;start=1961&amp;view=chart">Dinamarca</a></div><div>2023</div><div>-3,8</div><div>
'''Como vemos, encontramos los paises dentro de un div por el class name de country-name y los datos de año e inflacion en dos items div posteriormente'''
driver.get(url)
# XPath para encontrar todos los elementos con la clase 'item'
xpath_items = "//div[@class='item']"
items = driver.find_elements(By.XPATH, xpath_items)
data = []
for item in items:
    # Extraer el nombre del país
    pais = item.find_element(By.XPATH, ".//a[@class='country-name']").text.strip()

    # Extraer el año
    año = item.find_elements(By.TAG_NAME, "div")[1].text.strip() 

    # Extraer el valor de la inflación
    inflacion = item.find_elements(By.TAG_NAME, "div")[2].text.strip() 

    data.append({"País": pais, "Año": año, "Inflación": inflacion})

# Crear un DataFrame de pandas
import pandas as pd
df_inflacion = pd.DataFrame(data)
print(df_inflacion)
# Crear la tabla si no existe
conn = sqlite3.connect('C:/GitHubRepos/ProyectoFinal/data/processed/Dataset9k.db')
conn.execute( ''' CREATE TABLE IF NOT EXISTS EuroInfl (
        Pais TEXT,
        Año INTEGER,
        Inflacion REAL
    )
''')
#Introducimos el dataframe creado dentro de nuestra base de datos
df_inflacion.to_sql('EuroInfl', conn, if_exists='replace', index=False)

# Cerrar la conexión a la base de datos
conn.close()
del df_inflacion

In [5]:
#El PIB per cápita es el resultado de una interacción compleja de factores económicos que impactan directamente en el nivel de ingresos promedio de la población. 
#Productividad: El nivel de productividad es crucial para el PIB per cápita. Cuando la producción por trabajador aumenta, se genera más riqueza.
#https://datos.bancomundial.org/indicador/NY.GDP.PCAP.CD?locations=EU&most_recent_value_desc=false
driver = webdriver.Chrome()
url = "https://datos.bancomundial.org/indicador/NY.GDP.PCAP.CD?locations=EU&most_recent_value_desc=false"
driver.get(url)
xpath_items = "//div[@class='item']"
items = driver.find_elements(By.XPATH, xpath_items)
data = []
for item in items:
    
    pais = item.find_element(By.CSS_SELECTOR, "a.country-name").text.strip()
    año, pib_per_capita = item.find_elements(By.TAG_NAME, "div")[1:3]
    año = año.text.strip()
    pib_per_capita = pib_per_capita.text.strip()
    data.append({"País": pais, "Año": año, "PPC": pib_per_capita})
df_ppc = pd.DataFrame(data)
driver.quit()
conn = sqlite3.connect('C:/GitHubRepos/ProyectoFinal/data/processed/Dataset9k.db')
conn.execute( ''' CREATE TABLE IF NOT EXISTS EuroInfl (
        País TEXT,
        Año INTEGER,
        PPC REAL
    )
''')
#Introducimos el dataframe creado dentro de nuestra base de datos
df_ppc.to_sql('EuroPPC', conn, if_exists='replace', index=False)

# Cerrar la conexión a la base de datos
conn.close()
print(df_ppc)
del df_ppc

In [6]:
#EL SBMM o Sueldo Bruto Mensual Medio, otro indicador mas para agrupar y entender mejor los datos encuestados y tener un enfoque mas adecuado de los
#perfiles de las personas.
'''from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()
url = "https://www.idealista.com/news/finanzas/laboral/2023/08/21/807629-el-salario-medio-espanol-es-un-20-inferior-al-de-la-ue-asi-esta-en-cada-pais"
driver.get(url)
accept_cookies_button = driver.find_element(By.XPATH, "//button[text()='Aceptar y continuar']")
accept_cookies_button.click()
try:
    button = driver.find_element(By.ID, "didomi-notice-agree-button")
    button.click()
    print("Se ha hecho clic en el botón de aceptar cookies")
except Exception as e:
    print(f"Error al encontrar o hacer clic en el botón: {e}")
    
    
xpath_rows = "//tbody/tr[@class='datawrapper-GqFmZ-1c5djey svelte-1smti70']"
rows = driver.find_elements(By.XPATH, xpath_rows)

data = []

for row in rows:
    
    pais_element = row.find_element(By.CSS_SELECTOR, "th.first")
    pais = pais_element.text.strip()

    
    sbmm_element = row.find_element(By.CSS_SELECTOR, "td.last")
    sbmm = sbmm_element.text.strip()

    data.append({"País 2022": pais, "SBMM": sbmm})

df_sbmm = pd.DataFrame(data)
print(df_sbmm)
driver.quit()
conn = sqlite3.connect('C:/GitHubRepos/ProyectoFinal/data/processed/Dataset9k.db')
conn.execute( CREATE TABLE IF NOT EXISTS EuroInfl (
        País TEXT,
        SBMM REAL
    )
)
df_sbmm.to_sql('EuroSBMM', conn, if_exists='replace', index=False)
conn.close()
print(df_sbmm)
del df_sbmm'''
#El idealista tiene antibots ademas de cookies que no se guardan son temporales, voy a robaar la tabla con la extension de chrome store llamada INSTANT DATA SCRAPPER y subire el csv manualmente.
df_sbmm = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/raw/MainDataProyect/SBMMEUW.csv')
conn = sqlite3.connect('C:/GitHubRepos/ProyectoFinal/data/processed/Dataset9k.db')
conn.execute( '''CREATE TABLE IF NOT EXISTS EuroSBMM (
        País TEXT,
        SBMM REAL
        )
        ''')
df_sbmm.to_sql('EuroSBMM', conn, if_exists='replace', index=False)
conn.close()
print(df_sbmm)
del df_sbmm

                    País  Salario medio (€/mes)
0        Luxemburgo :lu:                   3713
1         Dinamarca :dk:                   3573
2          Alemania :de:                   3148
3           Irlanda :ie:                   3051
4           Bélgica :be:                   3007
5           Austria :at:                   2964
6      Países Bajos :nl:                   2964
7         Finlandia :fi:                   2678
8            Suecia :se:                   2604
9           Francia :fr:                   2574
10         Media UE :eu:                   2194
11           Italia :it:                   2174
12           España :es:                   1822
13        Eslovenia :si:                   1503
14           Chipre :cy:                   1359
15            Malta :mt:                   1354
16  República Checa :cz:                   1208
17         Portugal :pt:                   1154
18          Estonia :ee:                   1123
19         Lituania :lt:                